In [ ]:
using Distances, LinearAlgebra, Random, Statistics, Distributions
include("KernelQuantile.jl")
include("KernelUtils.jl")
include("KernelLogistic.jl")
include("KernelMultinomial.jl")

In [ ]:
p = 50
n = 16384
covM = ones(p,p)
for i in 1:p
    for j in 1:p
        covM[i,j] = 0.5^abs(i-j)
    end
end
λs = exp10.(range(1, -5, length=30))
ms = Int.(2 .^(range(3, 12, length=10)))
nreps = 10

In [ ]:
nreps = 10
qs = [range(3,10);]
m = 512
multinomial_errors1 = zeros(nreps,length(qs))
multinomial_errors2 = zeros(nreps,length(qs))
multinomial_errors_full1 = zeros(nreps,length(qs))
multinomial_errors_full2 = zeros(nreps,length(qs))
multinomial_errors_linear = zeros(nreps,length(qs))
multinomial_times_MM1 = zeros(nreps,length(qs))
multinomial_times_newton1 = zeros(nreps,length(qs))
multinomial_times_MM2 = zeros(nreps,length(qs))
multinomial_times_newton2 = zeros(nreps,length(qs))
multinomial_times_full1 = zeros(nreps,length(qs))
multinomial_times_full2 = zeros(nreps,length(qs))
for i in 1:nreps
    d = MvNormal(zeros(p), covM)
    X = Matrix(Transpose(rand(d,n)))
    ηs1 = -4 .+ sin.(X[:,1]).+ X[:,2] .* X[:, 3]  .+ X[:,4].^3 .- abs.(X[:,5]) .+0.1*vec(mapslices(norm, X[:,6:p]; dims=2).^2)
    ηs2 = 4 .+ cos.(X[:,1]).+ X[:,2] .* X[:, 4]  .+ X[:,5].^3 .- abs.(X[:,3]) .-0.1*vec(mapslices(norm, X[:,6:p]; dims=2).^2)
    X_test = Matrix(Transpose(rand(d,Int(n/4))))
    ηs1_test = -4 .+ sin.(X_test[:,1]).+ X_test[:,2] .* X_test[:, 3]  .+ X_test[:,4].^3 .- abs.(X_test[:,5]) .+0.1*vec(mapslices(norm, X_test[:,6:p]; dims=2).^2)
    ηs2_test = 4 .+ cos.(X_test[:,1]).+ X_test[:,2] .* X_test[:, 4]  .+ X_test[:,5].^3 .- abs.(X_test[:,3]) .-0.1*vec(mapslices(norm, X_test[:,6:p]; dims=2).^2)
    row_sums_buffer = zeros(n)
    for j in 1:length(qs)
        q = qs[j]
        ηs = [ηs1 ηs2 zeros(n,q-3)] 
        ηs_test =  [ηs1_test ηs2_test zeros(Int(n/4),q-3)] 
        P = zeros(n,q)
        P_pred = zeros(Int(n/4),q)
        compute_probs_reduced!(P,ηs, row_sums_buffer)
        Y = zeros(n,q)
        for k in 1:n
            probs = P[k, :]
            d3 = Multinomial(1, probs)
            Y[k,:] .= rand(d3)
        end
        P = zeros(Int(n/4),q)
        compute_probs_reduced!(P,ηs_test, row_sums_buffer)
        Y_test = zeros(Int(n/4),q)
        for k in 1:Int(n/4)
            probs = P[k, :]
            d3 = Multinomial(1, probs)
            Y_test[k,:] .= rand(d3)
        end
        σ = 15.0
        K = rbf_kernel(X,σ)
        G = generate_nystrom_G(n,m)
        @time B1s, total_time1 = KernelMultinomial(Y,K,G,λs,ϵ=1e-4,verbose=false)
        @time B2s, total_time2 = KernelMultinomial_newton(Y,K,G,λs,ϵ=1e-4,verbose=false)
        best_metric = -Inf
        for k in 1:length(λs)
            compute_probs_reduced!(P_pred,predict_krr(X,X_test,G,B1s[k,:,:],σ), row_sums_buffer)
            best_metric = max(loglikelihood(Y_test, P_pred),best_metric)
        end
        multinomial_errors1[i,j] = best_metric
        multinomial_times_MM1[i,j] = total_time1
        multinomial_times_newton1[i,j] = total_time2
        @time Bs, total_time = KernelMultinomial_full(Y,K,G,λs,ϵ=1e-4,verbose=false)
        best_metric = -Inf
        for k in 1:length(λs)
            compute_probs!(P_pred,predict_krr(X,X_test,G,Bs[k,:,:],σ), row_sums_buffer)
            best_metric = max(loglikelihood(Y_test, P_pred),best_metric)
        end
        multinomial_errors_full1[i,j] = best_metric
        multinomial_times_full1[i,j] = total_time
        σ = 30.0
        K = rbf_kernel(X,σ)
        @time B1s, total_time1 = KernelMultinomial(Y,K,G,λs,ϵ=1e-4,verbose=false)
        @time B2s, total_time2 = KernelMultinomial_newton(Y,K,G,λs,ϵ=1e-4,verbose=false)
        best_metric = -Inf
        for k in 1:length(λs)
            compute_probs_reduced!(P_pred,predict_krr(X,X_test,G,B1s[k,:,:],σ), row_sums_buffer)
            best_metric = max(loglikelihood(Y_test, P_pred),best_metric)
        end
        multinomial_errors2[i,j] = best_metric
        multinomial_times_MM2[i,j] = total_time1
        multinomial_times_newton2[i,j] = total_time2
        @time Bs, total_time = KernelMultinomial_full(Y,K,G,λs,ϵ=1e-4,verbose=false)
        best_metric = -Inf
        for k in 1:length(λs)
            compute_probs!(P_pred,predict_krr(X,X_test,G,Bs[k,:,:],σ), row_sums_buffer)
            best_metric = max(loglikelihood(Y_test, P_pred),best_metric)
        end
        multinomial_errors_full2[i,j] = best_metric
        multinomial_times_full2[i,j] = total_time
        B_linear,_,_ = FastMultinomial_reduced([ones(size(X,1)) X],Y)
        P_pred = zeros(Int(n/4),q)
        compute_probs_reduced!(P_pred,[ones(size(X_test,1)) X_test]*B_linear, row_sums_buffer)
        multinomial_errors_linear[i,j] = loglikelihood(Y_test, P_pred)
    end
end

λ=10.0 sovled in 45 iterations.
λ=6.2101694189156165 sovled in 39 iterations.
λ=3.856620421163471 sovled in 45 iterations.
λ=2.3950266199874855 sovled in 49 iterations.
λ=1.4873521072935114 sovled in 54 iterations.
λ=0.9236708571873862 sovled in 56 iterations.
λ=0.5736152510448679 sovled in 60 iterations.
λ=0.3562247890262442 sovled in 59 iterations.
λ=0.2212216291070449 sovled in 55 iterations.
λ=0.1373823795883263 sovled in 65 iterations.
λ=0.0853167852417281 sovled in 67 iterations.
λ=0.05298316906283707 sovled in 69 iterations.
λ=0.03290344562312668 sovled in 68 iterations.
λ=0.020433597178569417 sovled in 72 iterations.
λ=0.01268961003167922 sovled in 79 iterations.
λ=0.007880462815669913 sovled in 85 iterations.
λ=0.004893900918477494 sovled in 88 iterations.
λ=0.0030391953823131978 sovled in 104 iterations.
λ=0.0018873918221350976 sovled in 81 iterations.
λ=0.0011721022975334804 sovled in 110 iterations.
λ=0.0007278953843983154 sovled in 103 iterations.
λ=0.00045203536563602454 

λ=0.00010826367338740541 sovled in 112 iterations.
λ=6.723357536499335e-5 sovled in 101 iterations.
λ=4.1753189365604006e-5 sovled in 132 iterations.
λ=2.592943797404667e-5 sovled in 109 iterations.
λ=1.610262027560939e-5 sovled in 119 iterations.
λ=1.0e-5 sovled in 99 iterations.
 41.807652 seconds (15.16 k allocations: 4.794 GiB, 3.11% gc time)
λ=10.0 sovled in 7 iterations.
λ=6.2101694189156165 sovled in 6 iterations.
λ=3.856620421163471 sovled in 6 iterations.
λ=2.3950266199874855 sovled in 6 iterations.
λ=1.4873521072935114 sovled in 6 iterations.
λ=0.9236708571873862 sovled in 6 iterations.
λ=0.5736152510448679 sovled in 6 iterations.
λ=0.3562247890262442 sovled in 6 iterations.
λ=0.2212216291070449 sovled in 6 iterations.
λ=0.1373823795883263 sovled in 6 iterations.
λ=0.0853167852417281 sovled in 6 iterations.
λ=0.05298316906283707 sovled in 6 iterations.
λ=0.03290344562312668 sovled in 6 iterations.
λ=0.020433597178569417 sovled in 6 iterations.
λ=0.01268961003167922 sovled in 

15 32685.786404743896 15
16 32685.595159281576 16
17 32685.43502733125 17
18 32685.27915178939 18
19 32685.13017766495 19
20 32685.0078538853 20
21 32684.930677466516 21
22 32684.902252870674 22
λ=10.0 sovled in 34 iterations.
λ=6.2101694189156165 sovled in 33 iterations.
λ=3.856620421163471 sovled in 32 iterations.
λ=2.3950266199874855 sovled in 37 iterations.
λ=1.4873521072935114 sovled in 38 iterations.
λ=0.9236708571873862 sovled in 38 iterations.
λ=0.5736152510448679 sovled in 41 iterations.
λ=0.3562247890262442 sovled in 37 iterations.
λ=0.2212216291070449 sovled in 45 iterations.
λ=0.1373823795883263 sovled in 48 iterations.
λ=0.0853167852417281 sovled in 41 iterations.
λ=0.05298316906283707 sovled in 53 iterations.
λ=0.03290344562312668 sovled in 57 iterations.
λ=0.020433597178569417 sovled in 47 iterations.
λ=0.01268961003167922 sovled in 50 iterations.
λ=0.007880462815669913 sovled in 48 iterations.
λ=0.004893900918477494 sovled in 50 iterations.
λ=0.0030391953823131978 sovle

λ=0.00017433288221999874 sovled in 72 iterations.
λ=0.00010826367338740541 sovled in 93 iterations.
λ=6.723357536499335e-5 sovled in 84 iterations.
λ=4.1753189365604006e-5 sovled in 93 iterations.
λ=2.592943797404667e-5 sovled in 90 iterations.
λ=1.610262027560939e-5 sovled in 85 iterations.
λ=1.0e-5 sovled in 88 iterations.
 24.755219 seconds (13.12 k allocations: 1.946 GiB, 3.69% gc time)
λ=10.0 sovled in 7 iterations.
λ=6.2101694189156165 sovled in 6 iterations.
λ=3.856620421163471 sovled in 6 iterations.
λ=2.3950266199874855 sovled in 6 iterations.
λ=1.4873521072935114 sovled in 6 iterations.
λ=0.9236708571873862 sovled in 6 iterations.
λ=0.5736152510448679 sovled in 6 iterations.
λ=0.3562247890262442 sovled in 6 iterations.
λ=0.2212216291070449 sovled in 6 iterations.
λ=0.1373823795883263 sovled in 6 iterations.
λ=0.0853167852417281 sovled in 6 iterations.
λ=0.05298316906283707 sovled in 6 iterations.
λ=0.03290344562312668 sovled in 7 iterations.
λ=0.020433597178569417 sovled in 7

15 25216.357783858643 7
λ=10.0 sovled in 43 iterations.
λ=6.2101694189156165 sovled in 42 iterations.
λ=3.856620421163471 sovled in 45 iterations.
λ=2.3950266199874855 sovled in 44 iterations.
λ=1.4873521072935114 sovled in 50 iterations.
λ=0.9236708571873862 sovled in 50 iterations.
λ=0.5736152510448679 sovled in 53 iterations.
λ=0.3562247890262442 sovled in 60 iterations.
λ=0.2212216291070449 sovled in 67 iterations.
λ=0.1373823795883263 sovled in 56 iterations.
λ=0.0853167852417281 sovled in 57 iterations.
λ=0.05298316906283707 sovled in 65 iterations.
λ=0.03290344562312668 sovled in 62 iterations.
λ=0.020433597178569417 sovled in 59 iterations.
λ=0.01268961003167922 sovled in 69 iterations.
λ=0.007880462815669913 sovled in 64 iterations.
λ=0.004893900918477494 sovled in 65 iterations.
λ=0.0030391953823131978 sovled in 101 iterations.
λ=0.0018873918221350976 sovled in 98 iterations.
λ=0.0011721022975334804 sovled in 98 iterations.
λ=0.0007278953843983154 sovled in 103 iterations.
λ=

...

λ=0.00010826367338740541 sovled in 85 iterations.
λ=6.723357536499335e-5 sovled in 87 iterations.
λ=4.1753189365604006e-5 sovled in 83 iterations.
λ=2.592943797404667e-5 sovled in 86 iterations.
λ=1.610262027560939e-5 sovled in 85 iterations.
λ=1.0e-5 sovled in 82 iterations.
 20.860992 seconds (12.64 k allocations: 1.828 GiB, 3.79% gc time)
λ=10.0 sovled in 7 iterations.
λ=6.2101694189156165 sovled in 6 iterations.
λ=3.856620421163471 sovled in 6 iterations.
λ=2.3950266199874855 sovled in 6 iterations.
λ=1.4873521072935114 sovled in 6 iterations.
λ=0.9236708571873862 sovled in 6 iterations.
λ=0.5736152510448679 sovled in 6 iterations.
λ=0.3562247890262442 sovled in 6 iterations.
λ=0.2212216291070449 sovled in 6 iterations.
λ=0.1373823795883263 sovled in 6 iterations.
λ=0.0853167852417281 sovled in 6 iterations.
λ=0.05298316906283707 sovled in 6 iterations.
λ=0.03290344562312668 sovled in 6 iterations.
λ=0.020433597178569417 sovled in 6 iterations.
λ=0.01268961003167922 sovled in 6 ite

λ=10.0 sovled in 42 iterations.
λ=6.2101694189156165 sovled in 41 iterations.
λ=3.856620421163471 sovled in 47 iterations.
λ=2.3950266199874855 sovled in 48 iterations.
λ=1.4873521072935114 sovled in 54 iterations.
λ=0.9236708571873862 sovled in 55 iterations.
λ=0.5736152510448679 sovled in 61 iterations.
λ=0.3562247890262442 sovled in 61 iterations.
λ=0.2212216291070449 sovled in 68 iterations.
λ=0.1373823795883263 sovled in 62 iterations.
λ=0.0853167852417281 sovled in 64 iterations.
λ=0.05298316906283707 sovled in 64 iterations.
λ=0.03290344562312668 sovled in 60 iterations.
λ=0.020433597178569417 sovled in 69 iterations.
λ=0.01268961003167922 sovled in 67 iterations.
λ=0.007880462815669913 sovled in 71 iterations.
λ=0.004893900918477494 sovled in 67 iterations.
λ=0.0030391953823131978 sovled in 73 iterations.
λ=0.0018873918221350976 sovled in 72 iterations.
λ=0.0011721022975334804 sovled in 106 iterations.
λ=0.0007278953843983154 sovled in 99 iterations.
λ=0.00045203536563602454 so

λ=0.020433597178569417 sovled in 4 iterations.
λ=0.01268961003167922 sovled in 4 iterations.
λ=0.007880462815669913 sovled in 4 iterations.
λ=0.004893900918477494 sovled in 4 iterations.
λ=0.0030391953823131978 sovled in 4 iterations.
λ=0.0018873918221350976 sovled in 4 iterations.
λ=0.0011721022975334804 sovled in 4 iterations.
λ=0.0007278953843983154 sovled in 4 iterations.
λ=0.00045203536563602454 sovled in 4 iterations.
λ=0.0002807216203941176 sovled in 4 iterations.
λ=0.00017433288221999874 sovled in 4 iterations.
λ=0.00010826367338740541 sovled in 3 iterations.
λ=6.723357536499335e-5 sovled in 3 iterations.
λ=4.1753189365604006e-5 sovled in 3 iterations.
λ=2.592943797404667e-5 sovled in 3 iterations.
λ=1.610262027560939e-5 sovled in 3 iterations.
λ=1.0e-5 sovled in 3 iterations.
366.780516 seconds (45.87 k allocations: 27.944 GiB, 0.52% gc time)
λ=10.0 sovled in 49 iterations.
λ=6.2101694189156165 sovled in 39 iterations.
λ=3.856620421163471 sovled in 50 iterations.
λ=2.395026619

λ=0.0002807216203941176 sovled in 105 iterations.
λ=0.00017433288221999874 sovled in 126 iterations.
λ=0.00010826367338740541 sovled in 130 iterations.
λ=6.723357536499335e-5 sovled in 121 iterations.
λ=4.1753189365604006e-5 sovled in 130 iterations.
λ=2.592943797404667e-5 sovled in 122 iterations.
λ=1.610262027560939e-5 sovled in 119 iterations.
λ=1.0e-5 sovled in 119 iterations.
 39.090738 seconds (15.40 k allocations: 4.888 GiB, 3.14% gc time)
λ=10.0 sovled in 7 iterations.
λ=6.2101694189156165 sovled in 6 iterations.
λ=3.856620421163471 sovled in 6 iterations.
λ=2.3950266199874855 sovled in 6 iterations.
λ=1.4873521072935114 sovled in 6 iterations.
λ=0.9236708571873862 sovled in 6 iterations.
λ=0.5736152510448679 sovled in 6 iterations.
λ=0.3562247890262442 sovled in 6 iterations.
λ=0.2212216291070449 sovled in 6 iterations.
λ=0.1373823795883263 sovled in 6 iterations.
λ=0.0853167852417281 sovled in 6 iterations.
λ=0.05298316906283707 sovled in 6 iterations.
λ=0.03290344562312668 s

14 32896.77412000503 14
15 32896.522362935175 15
16 32896.36822121794 16
17 32896.23940520982 17
18 32896.10265226627 18
19 32895.96157958028 19
20 32895.840442932255 20
21 32895.762090643926 21
22 32895.73259799407 22


In [ ]:
using DelimitedFiles
writedlm("multinomial_times_MM1_q.csv",multinomial_times_MM1,",")
writedlm("multinomial_times_newton1_q.csv",multinomial_times_newton1,",")
writedlm("multinomial_times_MM2_q.csv",multinomial_times_MM2,",")
writedlm("multinomial_times_newton2_q.csv",multinomial_times_newton2,",")
writedlm("multinomial_times_full1_q.csv",multinomial_times_full1,",")
writedlm("multinomial_times_full2_q.csv",multinomial_times_full2,",")
writedlm("multinomial_errors1_q.csv",multinomial_errors1,",")
writedlm("multinomial_errors2_q.csv",multinomial_errors2,",")
writedlm("multinomial_errors_linear_q.csv",multinomial_errors_linear,",")
writedlm("multinomial_errors_full1_q.csv", multinomial_errors_full1, ",")
writedlm("multinomial_errors_full2_q.csv", multinomial_errors_full2, ",")

In [ ]:
using Plots
using Measures
using Statistics

# Example data for demonstration (replace with your actual data)
qs = 3:10
# Create a 3x1 plot layout
plot(layout = (1,2),size=(1200,600), margin=5mm)

plot!(qs, mean(multinomial_times_MM1,dims=1)[:], yerr = std(multinomial_times_MM1,dims=1)[:], label = "Quadratic MM (standard)", xlabel = "q", ylabel = "Time (seconds)", title = "sigma=15",subplot=1,color="red",markerstrokecolor=:red)
plot!(qs, mean(multinomial_times_newton1,dims=1)[:], yerr = std(multinomial_times_newton1,dims=1)[:], label = "Newton", subplot=1,color="blue",markerstrokecolor=:blue)
plot!(qs, mean(multinomial_times_full1,dims=1)[:], yerr = std(multinomial_times_full1,dims=1)[:], label = "Quadratic MM (full)", xlabel = "q", ylabel = "Time (seconds)", title = "sigma=15",subplot=1,color="green",markerstrokecolor=:green)

plot!(qs, mean(multinomial_times_MM2,dims=1)[:], yerr = std(multinomial_times_MM2,dims=1)[:], label = "Quadratic MM (standard)", xlabel = "q", ylabel = "Time (seconds)",title = "sigma=30",subplot=2,color="red",markerstrokecolor=:red)
plot!(qs, mean(multinomial_times_newton2,dims=1)[:], yerr = std(multinomial_times_newton2,dims=1)[:], label = "Newton", subplot=2,color="blue",markerstrokecolor=:blue, legend=:none)
plot!(qs, mean(multinomial_times_full2,dims=1)[:], yerr = std(multinomial_times_full2,dims=1)[:], label = "Quadratic MM (full)", xlabel = "q", ylabel = "Time (seconds)", title = "sigma=30",subplot=2,color="green",markerstrokecolor=:green)

savefig("increaseq.pdf")